In [1]:
import torch
from transformers import BartForConditionalGeneration
import pandas as pd 
from datasets import *

from sklearn.model_selection import train_test_split

/Users/kayhan/opt/anaconda3/envs/pendulum/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-20 23:30:45.548946: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Preprocessing

## Types of Experiments

1. gpt style prompting 
2. just concattenating reviews
3. thorough reviews 
4. adding more meta data to the prompts 


## Types of Models to Run Experiments on
1. BART base
2. GPT2 
3. BART Large
4. T5
5. Trying Distil models (ex: sshleifer/distilbart-cnn-12-6)

Other models:
BART, BigBird-Pegasus, Blenderbot, BlenderbotSmall, Encoder decoder, FairSeq Machine-Translation, GPTSAN-japanese, LED, LongT5, M2M100, Marian, mBART, MT5, MVP, NLLB, NLLB-MOE, Pegasus, PEGASUS-X, PLBart, ProphetNet, SwitchTransformers, T5, XLM-ProphetNet


## Experiment 1. Adding gpt style prompting for BART with the following structure:

"The following are the highest rated reviews of a restaurant on Yelp. Generate a summary that details the opinions about this restaurant:

1. 'Great experience at this new 3.5 month old Korean/Thai fusion gem that offered delicious food, warm, friendly service, and a nice comfortable ambiance. Adam, our server, was very welcoming, fun, engaging, helpful with menu recommendations, and thoughtful to check in on us regularly. The items we ordered were all wonderful and flavorful - the hamachi Kama, Korean spicy noodle with pork jowl, Bibimbap with beef brisket, lavender lemonade, and apple pie cheese cake. Do yourself a favor and ask for their house made chili crisp oil and sauces - so so good. Go and enjoy a great vibe, meal, and environment.'
2. 'Everything here is excellent. My favorites are the Beef Noodle Soup, Korean Spicy Noodle and the newly added salmon green curry. Great gluten-free options too!'
3. ....
4. ...
5. ...

We do the first 5 reviews instead of 10 because the model only has a max length


## Experiment 2. ADdding more meta data to prompts 

Review Set:

Restaurant Name: The Blue Door

Location: San Francisco, CA

Cuisine: American

Review 1: "The Blue Door is a must-visit if you're in San Francisco. The food is outstanding, with a great selection of American classics and some unique dishes. The service is also top-notch, with friendly and attentive staff."

Review 2: "I was blown away by the food at The Blue Door. Every dish we tried was amazing, and the drinks were excellent too. The service was also great - our server was very knowledgeable about the menu and made some great recommendations."

Review 3: "I've been to The Blue Door several times now and it never disappoints. The food is consistently excellent and the service is always friendly and attentive. I highly recommend the fried chicken and the mac and cheese."

Review 4: "The Blue Door has some of the best food I've ever had. The flavors are so well-balanced and everything is cooked to perfection. The service is also excellent - our server was very friendly and made us feel welcome."

Review 5: "This place is a hidden gem. The food is outstanding and the atmosphere is cozy and inviting. I highly recommend the beef brisket and the apple pie."

Prompt:
"Summarize the top 5 reviews for The Blue Door in San Francisco, CA. The restaurant serves American cuisine and is known for its outstanding food and top-notch service."

In [16]:
path = '../data/summarized_reviews_san_francisco.csv'
df = pd.read_csv(path)

In [17]:
seperator = '_______JOINED_THE_MESSAGE_ON THIS_STRING_______'

In [22]:
def determine_prompt_type(review_type):
    if review_type == 'top_rated':
        top_rated_prompt = "The following are the highest rated reviews of a restaurant on Yelp. Generate a summary that details the opinions about this restaurant:\n\n"
        return top_rated_prompt
    
    elif review_type == "low_rated":
        low_rated_prompt = "The following are the lowest rated reviews of a restaurant on Yelp. Generate a summary that details the opinions about this restaurant:\n\n"
        return low_rated_prompt
    
    elif review_type == "newest_rated":
        newest_rated_prompt = "The following are the newest rated reviews of a restaurant on Yelp. Generate a summary that details the opinions about this restaurant\n\n"
        return newest_rated_prompt
    
    elif review_type == "elited_rated":
    
        elite_rated_prompt = "The following are restaurant Yelp reviews of a restaurant written by Yelp Elite members. Generate a summary that details the opinions about this restaurant:\n\n"
        return elite_rated_prompt
    
    else:
        return "The following are restaurant Yelp reviews of a restaurant. Generate a summary that details the opinions about this restaurant:\n\n"

In [28]:
def create_prompts(df):
    
    for i, row in df.iterrows():

        prompt = determine_prompt_type(row['review_type'])
        for i_, review in enumerate(row['review']):
            if i_ > 5:
                break
            prompt += f'{i+1}. {review} \n'
        df.loc[i,'prompt'] = prompt

    return df 

In [30]:
df = create_prompts(df)

In [40]:
df_train, df_test = train_test_split(df[['name','review_type','prompt','summary']]],test_size=0.2,random_state=0)


In [42]:
train_dataset = Dataset.from_pandas(df_train,split='train')
test_dataset = Dataset.from_pandas(df_test,split='test')
ds_dict = {'train':train_dataset,'test':test_dataset}
dataset = DatasetDict(ds_dict)
dataset

DatasetDict({
    train: Dataset({
        features: ['name', 'review_type', 'prompt', 'summary', '__index_level_0__'],
        num_rows: 2822
    })
    test: Dataset({
        features: ['name', 'review_type', 'prompt', 'summary', '__index_level_0__'],
        num_rows: 706
    })
})

In [44]:
import evaluate

rouge = evaluate.load("rouge")

In [51]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np


In [48]:
checkpoint = 'facebook/bart-base'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [53]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["prompt"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=150, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [54]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

100%|██████████| 1/1 [00:00<00:00,  2.18ba/s]


In [55]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [57]:
dataset

DatasetDict({
    train: Dataset({
        features: ['name', 'review_type', 'prompt', 'summary', '__index_level_0__'],
        num_rows: 2822
    })
    test: Dataset({
        features: ['name', 'review_type', 'prompt', 'summary', '__index_level_0__'],
        num_rows: 706
    })
})

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="BART_v2",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
tuned_model = trainer.model